# 📈 Model Değerlendirme & Analiz
Final modelin değerlendirme metrikleri ve görselleştirmeler.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor

In [ ]:
train = pd.read_csv('../data/train.csv')
stores = pd.read_csv('../data/stores.csv')
features = pd.read_csv('../data/features.csv')
train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])

In [ ]:
# Data Merge & Features
data = train.merge(stores, on='Store', how='left')
data = data.merge(features, on=['Store','Date','IsHoliday'], how='left')
markdown_cols = ['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']
data[markdown_cols] = data[markdown_cols].fillna(0)
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Week'] = data['Date'].dt.isocalendar().week.astype(int)
data['Quarter'] = data['Date'].dt.quarter
data['DayOfYear'] = data['Date'].dt.dayofyear
le = LabelEncoder()
data['Type'] = le.fit_transform(data['Type'])

## 🎯 Metrik Hesaplama

In [ ]:
y = data['Weekly_Sales']
X = data.drop(['Weekly_Sales','Date'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae, rmse

## 🌟 Feature Importance Grafik

In [ ]:
importance = model.feature_importances_
features_names = X.columns
imp_df = pd.DataFrame({'Feature': features_names, 'Importance': importance})
imp_df.sort_values(by='Importance', ascending=False).head()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x='Importance', y='Feature', data=imp_df.sort_values(by='Importance', ascending=False))
plt.title('Feature Importance')
plt.tight_layout()
plt.savefig('../docs/feature_importance.png')
plt.show()

📌 Grafik docs klasörüne kaydedilmiştir!